## BaseLine for bert-project

### Procedure
1. convert the `dev.json` to four sentences: (question, correct answer), (question, wrong answer). 
For instance, original sentence is **"Tommy glided across the marble floor with ease, but slipped and fell on the wet floor because `_____` has more resistance. (A) marble floor (B) wet floor"** where correct answer is wet floor. We can convert this sentenct into: (**"Tommy glided across the marble floor with ease, but slipped and fell on the wet floor because `_____` has more resistance."**, **"wet floor"**), (**"Tommy glided across the marble floor with ease, but slipped and fell on the wet floor because `_____` has more resistance."**, **"marble floor"**)
2. using bert with pre-trained model `bert-base-uncased` to predict the next sentence. In this case, we use correct and wrong answer as the next sentence to get confidence separately, pick up the higer one as the predicted answer.
3. accuracy on dev is `55.4 %`, on test is `53.3 %`

In [50]:
import json
import pprint
import collections
import re
import pprint

from os.path import abspath, dirname, join

strip_space = lambda s: re.sub("\s+", " ", s.strip().replace("\r", "").replace("\n", ""))
BASE_PATH = "/Users/shenweihai/Desktop/Projects/Ubuntu_exx/tutorial/bert-proj"

def load_examples(file_name: str):
    ans = []
    err = collections.defaultdict(int)
    with open(file_name, 'r') as handler:
        for line in handler:
            example = json.loads(line)
            question = example['question']
            example = json.loads(line)
            question = example['question']
            
            idx = question.index("(A)")
            Q, second = strip_space(question[0:idx]), question[idx:]
                
            ans_idx = second.index("(B)")
            ans_first, ans_second = strip_space(second[0:ans_idx].replace("(A)", "")), strip_space(second[ans_idx:].replace("(B)", ""))
            
            if example.get("answer_index", 0) == 1:
                ans_first, ans_second = ans_second, ans_first
            
            ans.append((Q, ans_first, ans_second))
    return ans

ans = load_examples(join(BASE_PATH, "quarel-data", "quarel-v1-dev.json"))
ans = load_examples(join(BASE_PATH, "quarel-data", "quarel-v1-test.json"))
pprint.pprint(ans[0:3])

[('Does the Sun appear dimmer on Earth or Pluto?', 'Pluto', 'Earth'),
 ('Tank the kitten learned from trial and error that carpet is rougher then '
  'skin. When he scratches his claws over carpet it generates _____ then when '
  'he scratches his claws over skin',
  'more heat',
  'less heat'),
 ('An airplane takes longer to reach takeoff speed on a dirt runway than on a '
  'paved runway. This is because the dirt runway has',
  'more resistance or',
  'less resistance.')]


In [51]:
# get corresponding text and segments_ids
import json
from pytorch_pretrained_bert import BertTokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

bert_ans = []
for Q, correct, wrong in ans:
    first_sentence = tokenizer.tokenize("[CLS] {0} [SEP]".format(Q)) 
    
    tokenized_text = tokenizer.tokenize("[CLS] {0} [SEP] {1} [SEP]".format(Q, correct))
    seg_ids = [1] * len(tokenized_text)
    for idx in range(len(first_sentence)):
        seg_ids[idx] = 0
    
    tokenized_text2 = tokenizer.tokenize("[CLS] {0} [SEP] {1} [SEP]".format(Q, wrong))
    seg_ids2 = [1] * len(tokenized_text2)
    for idx in range(len(first_sentence)):
        seg_ids2[idx] = 0
        
    bert_ans.append(["[CLS] {0} [SEP] {1} [SEP]".format(Q, correct), json.dumps(seg_ids), "[CLS] {0} [SEP] {1} [SEP]".format(Q, wrong), json.dumps(seg_ids2)])

pprint.pprint(bert_ans[0:3])


[['[CLS] Does the Sun appear dimmer on Earth or Pluto? [SEP] Pluto [SEP]',
  '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]',
  '[CLS] Does the Sun appear dimmer on Earth or Pluto? [SEP] Earth [SEP]',
  '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]'],
 ['[CLS] Tank the kitten learned from trial and error that carpet is rougher '
  'then skin. When he scratches his claws over carpet it generates _____ then '
  'when he scratches his claws over skin [SEP] more heat [SEP]',
  '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '
  '0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]',
  '[CLS] Tank the kitten learned from trial and error that carpet is rougher '
  'then skin. When he scratches his claws over carpet it generates _____ then '
  'when he scratches his claws over skin [SEP] less heat [SEP]',
  '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '
  '0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]'],
 ['[CLS] An airpla

In [52]:
# try the predict next sentence model
# https://github.com/huggingface/pytorch-pretrained-BERT/issues/48
import json
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM,BertForNextSentencePrediction,BertForSequenceClassification
import torch.nn.functional as F

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Load pre-trained model (weights)
model = BertForNextSentencePrediction.from_pretrained(model_name)
model.eval()

total, total_suc = 0, 0
# Tokenized input
for item in bert_ans:
    text, segs, text_err, segs_err =  item

    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = json.loads(segs)

    tokenized_text_err = tokenizer.tokenize(text_err)
    indexed_tokens_err = tokenizer.convert_tokens_to_ids(tokenized_text_err)
    segments_ids_err = json.loads(segs_err)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    tokens_tensor_err = torch.tensor([indexed_tokens_err])
    segments_tensors_err = torch.tensor([segments_ids_err])

    # # Load pre-trained model (weights)
    # model = BertForNextSentencePrediction.from_pretrained(model_name)
    # model.eval()

    # Predict is Next Sentence ?
    predictions = model(tokens_tensor, segments_tensors )
    confidence = float(F.softmax(predictions)[0][0])

    predictions_err = model(tokens_tensor_err, segments_tensors_err)
    confidence_err = float(F.softmax(predictions_err)[0][0])

    total += 1
    if confidence > confidence_err:
        total_suc += 1
    print("current acc: %s, total_suc: %s, total: %s" % (total_suc / (total + 0.0), total_suc, total))
    # print(confidence, confidence_err)


/Users/shenweihai/anaconda3/envs/py_37/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/shenweihai/anaconda3/envs/py_37/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


current acc: 1.0, total_suc: 1, total: 1
current acc: 0.5, total_suc: 1, total: 2
current acc: 0.3333333333333333, total_suc: 1, total: 3
current acc: 0.25, total_suc: 1, total: 4
current acc: 0.2, total_suc: 1, total: 5
current acc: 0.16666666666666666, total_suc: 1, total: 6
current acc: 0.2857142857142857, total_suc: 2, total: 7
current acc: 0.375, total_suc: 3, total: 8
current acc: 0.3333333333333333, total_suc: 3, total: 9
current acc: 0.4, total_suc: 4, total: 10
current acc: 0.36363636363636365, total_suc: 4, total: 11
current acc: 0.4166666666666667, total_suc: 5, total: 12
current acc: 0.46153846153846156, total_suc: 6, total: 13
current acc: 0.42857142857142855, total_suc: 6, total: 14
current acc: 0.4666666666666667, total_suc: 7, total: 15
current acc: 0.5, total_suc: 8, total: 16
current acc: 0.5294117647058824, total_suc: 9, total: 17
current acc: 0.5555555555555556, total_suc: 10, total: 18
current acc: 0.5263157894736842, total_suc: 10, total: 19
current acc: 0.55, tot

current acc: 0.54421768707483, total_suc: 80, total: 147
current acc: 0.5472972972972973, total_suc: 81, total: 148
current acc: 0.5503355704697986, total_suc: 82, total: 149
current acc: 0.5466666666666666, total_suc: 82, total: 150
current acc: 0.5496688741721855, total_suc: 83, total: 151
current acc: 0.5526315789473685, total_suc: 84, total: 152
current acc: 0.5555555555555556, total_suc: 85, total: 153
current acc: 0.5584415584415584, total_suc: 86, total: 154
current acc: 0.5548387096774193, total_suc: 86, total: 155
current acc: 0.5576923076923077, total_suc: 87, total: 156
current acc: 0.554140127388535, total_suc: 87, total: 157
current acc: 0.5506329113924051, total_suc: 87, total: 158
current acc: 0.5534591194968553, total_suc: 88, total: 159
current acc: 0.55625, total_suc: 89, total: 160
current acc: 0.5527950310559007, total_suc: 89, total: 161
current acc: 0.5493827160493827, total_suc: 89, total: 162
current acc: 0.5460122699386503, total_suc: 89, total: 163
current acc

current acc: 0.4842105263157895, total_suc: 138, total: 285
current acc: 0.486013986013986, total_suc: 139, total: 286
current acc: 0.4843205574912892, total_suc: 139, total: 287
current acc: 0.4861111111111111, total_suc: 140, total: 288
current acc: 0.4844290657439446, total_suc: 140, total: 289
current acc: 0.4862068965517241, total_suc: 141, total: 290
current acc: 0.4845360824742268, total_suc: 141, total: 291
current acc: 0.4863013698630137, total_suc: 142, total: 292
current acc: 0.4880546075085324, total_suc: 143, total: 293
current acc: 0.48639455782312924, total_suc: 143, total: 294
current acc: 0.4847457627118644, total_suc: 143, total: 295
current acc: 0.4831081081081081, total_suc: 143, total: 296
current acc: 0.48148148148148145, total_suc: 143, total: 297
current acc: 0.48322147651006714, total_suc: 144, total: 298
current acc: 0.48494983277591974, total_suc: 145, total: 299
current acc: 0.4866666666666667, total_suc: 146, total: 300
current acc: 0.4883720930232558, tota

current acc: 0.508274231678487, total_suc: 215, total: 423
current acc: 0.5094339622641509, total_suc: 216, total: 424
current acc: 0.5105882352941177, total_suc: 217, total: 425
current acc: 0.5117370892018779, total_suc: 218, total: 426
current acc: 0.5128805620608899, total_suc: 219, total: 427
current acc: 0.5116822429906542, total_suc: 219, total: 428
current acc: 0.5128205128205128, total_suc: 220, total: 429
current acc: 0.513953488372093, total_suc: 221, total: 430
current acc: 0.5150812064965197, total_suc: 222, total: 431
current acc: 0.5162037037037037, total_suc: 223, total: 432
current acc: 0.5150115473441108, total_suc: 223, total: 433
current acc: 0.5161290322580645, total_suc: 224, total: 434
current acc: 0.5172413793103449, total_suc: 225, total: 435
current acc: 0.5160550458715596, total_suc: 225, total: 436
current acc: 0.517162471395881, total_suc: 226, total: 437
current acc: 0.5159817351598174, total_suc: 226, total: 438
current acc: 0.5148063781321185, total_suc: